In [1]:
!gdown "https://drive.google.com/uc?id=1UrFtHryRVNPhsJgIscgs_K9DZZRy1KM7"

Downloading...
From: https://drive.google.com/uc?id=1UrFtHryRVNPhsJgIscgs_K9DZZRy1KM7
To: /content/salaries.json
100% 2.42M/2.42M [00:00<00:00, 158MB/s]


In [2]:
import json
from functools import reduce

In [3]:
# Função para carregar os dados
def carrega_dados(path:str) -> list[dict]:
    try:
        with open(path, 'r') as arquivo: # abre o arquivo especificado pelo caminho path no modo de leitura ('r')
            dados = arquivo.read()
            return json.loads(dados)
    except FileNotFoundError: # ocorre quando o arquivo especificado em open() não é encontrado
        return []

Nível de Experiência: Os níveis de experiência variam de "EX" (Experiente) a "MI" (Muito Inexperiente) e "SE" (Sênior). Isso indica o nível de experiência dos funcionários.

Tipo de Emprego: Todos os registros têm "FT" (Tempo Integral) como tipo de emprego, o que significa que esses funcionários trabalham em período integral.

## Estudando os dados:

In [30]:
# visualizando um único dado para compreender o formato
dados = carrega_dados("salaries.json")
dados[0]

{'work_year': '2023',
 'experience_level': 'EX',
 'employment_type': 'FT',
 'job_title': 'Data Science Director',
 'salary': '212000',
 'salary_currency': 'USD',
 'salary_in_usd': '212000',
 'employee_residence': 'US',
 'remote_ratio': 0,
 'company_location': 'US',
 'company_size': 'M'}

In [31]:
for i, item in enumerate(dados, start=1):
  item['id'] = i

In [32]:
len(dados) # quantidade de dados (dicionários)

8805

In [33]:
# Entendendo a quais anos pertencem os dados
anos_unicos = list(set(dado['work_year'] for dado in dados))
print(anos_unicos)

['2023', '2021', '2022', '2020']


In [34]:
%%timeit
# AQUI criar função para remover os dados que não vamos considerar (experience_level, employment_type, salary_currency, remote_ratio)
def remover_info(json, *chaves) -> list[dict]:
    try:
        return list(map(lambda d: {k: v for k, v in d.items() if k not in chaves}, json))
    except KeyError as e:
        raise ValueError(f'Erro ao remover informações: {e}')

chave_a_remover = 'salary_currency' # posso passar uma lista com as chaves a serem removidas para a função
novos_dados = remover_info(dados, chave_a_remover)

#print(novos_dados[0])

22.6 ms ± 5.16 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Realizando a atividade proposta:

In [98]:
# filtrando por ano -> a saída de cada função será a entrada da próxima
def filtro_ano(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['work_year'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')
# |
# v

# filtra por nível de experiencia:
def filtro_experiencia(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['experience_level'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')

# |
# v

# filtra por tipo de cargo
def filtro_cargo(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['job_title'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')

# |
# v

# filtra por salário -> EM DÓLAR
def filtro_salario(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['salary_in_usd'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')

# |
# v

# filtra por localização do funcionário
def filtro_localizacoes_funcionario(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['employee_residence'] == opc]
    except TypeError:
        raise ValueError('Filtro de localização inválido .')
    except Exception as e:
        raise ValueError(f'Erro no filtro de localização funcionário: {e}')
# |
# v

# filtra por localização da empresa
def filtro_localizacoes_empresa(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['company_location'] == opc]
    except TypeError:
        raise ValueError('Filtro de localização Empresa inválido .')
    except Exception as e:
        raise ValueError(f'Erro no filtro localização Empresa: {e}')

# |
# v

# filtra por tamanho da empresa
def filtro_porte_empresa(item, opc) -> list[dict]:
    try:
        return [dado for dado in item if dado['company_size'] == opc]
    except TypeError:
        raise ValueError('Filtro de ano inválido.')
    except Exception as e:
        raise ValueError(f'Erro no filtro de ano: {e}')

In [28]:
def localizacoes(dados, chave = str):
    # Retorna uma lista das localizações únicas dos funcionários, obtidas a partir da chave 'company_location' de cada funcionário
    return list(set(funcionario.get(chave) for funcionario in dados if funcionario.get(chave)))

In [99]:
localizacao_funcionario = localizacoes(dados, 'employee_residence')
localizacao_empresa = localizacoes(dados, 'company_location')

In [142]:
# Opções para o menu
cargos = {
    'Tech Lead': 'Data Science Tech Lead',
    'Practitioner': 'Data Science Practitioner',
    'Engineer': 'Data Science Engineer',
    'Consultant': 'Data Science Consultant',
    'Manager': 'Data Science Manager',
    'Managing Director': 'Managing Director Data Science',
    'Director': 'Data Science Director',
    'Lead': 'Data Science Lead',
    'Head': 'Head of Data Science'
}

experiencias = {'SE': 'Senior', 'MI': 'Analista', 'EN': 'Junior', 'EX': 'Executivo'}
tamanhos = {'S': 'Pequena', 'M': 'Media', 'L': 'Grande'}
anos = ['2020', '2021', '2022', '2023']

In [101]:
def obter_ano(anos, msg='Anos') -> str:
    while True:
        try:
            ano = input(f"{msg} ({' | '.join(anos)}): ")
            if ano in anos:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')

    return ano

In [102]:
def obter_localizacao_empresa(localizacoes_empresas, msg='Sedes das empresas') -> str:
    while True:
        try:
            localizacao_empresa = input(f"{msg} ({' | '.join(localizacoes_empresas)}): ").upper()

            if localizacao_empresa in localizacoes_empresas:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')

    return localizacao_empresa

In [7]:
def obter_localizacao_funcionario(localizacoes_funcionarios, msg='Residências funcionários') -> str:
    while True:
        try:
            local_funcionario = input(f"{msg} ({' | '.join(localizacoes_funcionarios)}): ").upper()

            if local_funcionario in localizacoes_funcionarios:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')

    return local_funcionario

In [ ]:
x = obter_localizacao_funcionario(localizacao_funcionario)

employee_residence (CR | AT | HN | IR | NG | GE | BA | ZA | IT | JE | PR | RS | SG | GB | IN | EC | EE | NZ | PL | TH | US | FI | NO | AR | BR | BE | MU | SA | SE | CA | BG | AU | MX | ES | LT | CF | PK | DZ | IL | MY | UA | RU | CN | SI | GH | PH | NL | AE | CY | AM | KE | UG | GR | HK | HU | IE | CO | KR | DK | LU | QA | IQ | LV | RO | DE | AD | CH | MD | ID | PE | JP | BO | KW | TR | FR | VN | UZ | MT | EG | HR | CL | DO | PT | CZ | AS | TN): tn


In [20]:
def obter_experiencia(experiencias, msg='Experiencia') -> str:
    while True:
        try:
            experiencia = input(f"{msg} {' | '.join(experiencias)}): ").upper()

            if experiencia in experiencias:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return experiencia

In [25]:
def obter_porte_empresa(tamanhos, msg='Tamanho') -> str:
    while True:
        try:
            tamanho = input(f"{msg} {' | '.join(tamanhos)}): ").upper()
            if tamanho in tamanhos:
                break
            else:
                raise ValueError('Entrada Inválida! Dado não está na lista.')
        except ValueError as e:
            print(f'Erro: {e}\n')
    return tamanho

In [143]:
def obter_cargo(cargos, msg='Cargos de Data Science:'):
    while True:
        try:
            msg_prompt = f"{msg} ({' | '.join(cargos.keys())})"
            cargo_key = input(msg_prompt).title()

            if cargo_key in cargos:
                cargo = cargos[cargo_key]
                print(f"Valores associados ao cargo '{cargo_key}': {cargo}")
                break
            else:
                raise ValueError('Filtro de cargo inválido.')

        except ValueError as ve:
            raise ValueError(f'Erro no filtro de cargo: {ve}')

    return cargo


In [ ]:
def inserir_novo_dado(profissional: list[dict]) -> bool:
    profissional.append({
        'ano_de_trabalho': input('Qual o ano: \n'),
        'nivel_de_experiencia': input('Qual a experiência: \n'),
        'tipo_de_emprego': input('Qual o tipo: \n'),
        'cargo': input('Qual a profissão: \n'),
        'salario_em_usd': input('Qual o salário em dolar: \n'),
        'residencia_do_funcionario': input('Qual a residência : \n'),
        'localizacao_da_empresa': input('Qual a localização da empresa: \n'),
        'tamanho_da_empresa': input('Quantos funcionários: \n'),
        'id': len(dados) + 1
    })
    return True

In [ ]:
def apagar_dado(dados: list[dict]):
    id = int(input('Informe o id que deseja remover: '))
    apagado = [registro for registro in dados if registro['id'] == id]
    if len(apagado) == 0:
        print('Não foi encontrado!')
    else:
        print(f'Excluídos {len(apagado)} registros!')
    dados_novos = [dic for dic in dados if dic['id'] != id]
    return dados_novos

In [ ]:
def media_salario_ano(lista_personalizada, cargo: str) -> float:
    try:
        # Filtra os elementos da lista_personalizada com base na condição especificada pela função lambda
        # Retorna True para os itens; o filter devolve um iterador com apenas os itens correspondentes ao cargo desejado
        # Map aplica a função lambda em cada elemento da lista e transforma string em float
        salario_do_cargo = list(map(lambda item: float(item['salary_in_usd']), filter(lambda item: item['job_title'] == cargo, lista_personalizada)))

        if salario_do_cargo:
            media_salarial = sum(salario_do_cargo) / len(salario_do_cargo)
            return media_salarial
        else:
            raise ValueError('Nenhum salário encontrado para o cargo especificado.')
    except (KeyError, ValueError, ZeroDivisionError) as e:
        # Captura exceções específicas que podem ocorrer durante a execução da função
        # KeyError: Tentativa de acessar uma chave que não existe no dicionário
        # ValueError: Conversão de string para float falhou
        # ZeroDivisionError: Tentativa de divisão por zero
        raise ValueError(f'Erro ao calcular a média salarial: {e}')

media_salario_ano(dados, 'Head of Data Science')

178387.25

In [95]:
def menu(dados, msg='nome do programa', msg_2 = 'Insira um número'):
    print(f'{msg}\n')
    while True:
      opcoes = {'1': 'Verificar média salarial por cargo: ',
                '2': 'Verificar salário máximo por cargo: ',
                '3': 'Verificar salário mínimo por cargo: ',
                '4': 'Inserir dados',
                '5': 'Remover dados',
                '6': 'Atualizar dados',
                '0': 'Sair'}

      msg = f"{msg_2}({' | '.join([f'{key} - {values}' for key, values in opcoes.items()])}): "
      i = input(msg)


      if i == '1' or i == '2' or i == '3':
        ano = obter_ano(anos)
        dados_finais = filtro_ano(dados, ano)
        contagem = len(dados_finais)
        print(contagem)

        exp = obter_experiencia(experiencias, msg='Experiencia')
        dados_finais = filtro_experiencia(dados_finais, exp)
        contagem = len(dados_finais)
        print(contagem)

        cargo = obter_cargo(cargos)
        dados_finais = filtro_cargo(dados_finais, cargo)
        contagem = len(dados_finais)
        print(contagem)

        loc_func = obter_localizacao_funcionario(localizacao_funcionario)
        dados_finais = filtro_localizacoes_funcionario(dados_finais, loc_func)
        contagem = len(dados_finais)
        print(contagem)

        loc_emp = obter_localizacao_empresa(localizacao_empresa)
        dados_finais = filtro_localizacoes_empresa(dados_finais, loc_emp)
        contagem = len(dados_finais)
        print(contagem)

        porte = obter_porte_empresa(tamanhos, msg='Tamanho')
        dados_finais = filtro_porte_empresa(dados_finais, porte)
        contagem = len(dados_finais)
        print(contagem)

      elif i == '4':
        inserir_novo_dado(dados)

      elif i == '5':
        apagar_dado(dados)

      elif i == '6':
        pass

      elif i == '0':
        break

      else:
        print('Digite uma opção válida')
      return dados_finais

In [85]:
{'work_year': '2023',
 'experience_level': 'EX',
 'employment_type': 'FT',
 'job_title': 'Data Science Director',
 'salary': '212000',
 'salary_currency': 'USD',
 'salary_in_usd': '212000',
 'employee_residence': 'US',
 'remote_ratio': 0,
 'company_location': 'US',
 'company_size': 'M'}

{'work_year': '2023',
 'experience_level': 'EX',
 'employment_type': 'FT',
 'job_title': 'Data Science Director',
 'salary': '212000',
 'salary_currency': 'USD',
 'salary_in_usd': '212000',
 'employee_residence': 'US',
 'remote_ratio': 0,
 'company_location': 'US',
 'company_size': 'M'}

In [146]:
menu(dados)

nome do programa

Insira um número(1 - Verificar média salarial por cargo:  | 2 - Verificar salário máximo por cargo:  | 3 - Verificar salário mínimo por cargo:  | 4 - Inserir dados | 5 - Remover dados | 6 - Atualizar dados | 0 - Sair): 1
Anos (2020 | 2021 | 2022 | 2023): 2023
6861
Experiencia SE | MI | EN | EX): EX
214
Cargos de Data Science: (Tech Lead | Practitioner | Engineer | Consultant | Manager | Managing Director | Director | Lead | Head)Director
Valores associados ao cargo 'Director': Data Science Director
2
Residências funcionários (MT | AE | ID | AR | US | HK | HN | CY | LV | NO | MU | KW | DE | BR | BE | DO | DK | BO | HR | LU | NG | HU | VN | PK | SE | BG | DZ | SA | QA | EE | CH | TR | UA | EG | PL | CO | IE | UZ | TN | CN | MY | NL | LT | FI | FR | ES | IT | RO | RU | AM | UG | KE | GE | GR | PH | ZA | CR | CF | CL | NZ | BA | PR | JE | EC | PT | KR | AS | JP | CZ | SI | SG | GH | AT | GB | RS | IQ | IL | PE | MD | IR | CA | AU | TH | AD | IN | MX): US
2
Sedes das empre

[{'work_year': '2023',
  'experience_level': 'EX',
  'employment_type': 'FT',
  'job_title': 'Data Science Director',
  'salary': '212000',
  'salary_currency': 'USD',
  'salary_in_usd': '212000',
  'employee_residence': 'US',
  'remote_ratio': 0,
  'company_location': 'US',
  'company_size': 'M',
  'id': 1},
 {'work_year': '2023',
  'experience_level': 'EX',
  'employment_type': 'FT',
  'job_title': 'Data Science Director',
  'salary': '190000',
  'salary_currency': 'USD',
  'salary_in_usd': '190000',
  'employee_residence': 'US',
  'remote_ratio': 0,
  'company_location': 'US',
  'company_size': 'M',
  'id': 2}]

In [ ]:
def menu():
  pass
  # obter ano que deseja ver os dados - Isadora OK
  # obter nível de experiência que deseja saber -> Rafael OK
  # obter cargo -> Ingrid
  # obter localização do funcionário -> Eliane
  # obter localização da empresa - Eliane
  # obter tamanho da empresa -> Rafael

  # inserir dados -> Pedro OK
  # remover dados -> Pedro OK


  # retornar média salarial, maior salário ou menor salário

# obs: (maior salário, menor salário) -> deve ser obtido com uma função que retorna lista de tuplas

--------------------------------------------------------------------------

In [ ]:
# Ex de uso se fosse considerar apenas um filtro
def filtrar_dados(lista, filtro_funcao, opc=None):
    return list(filter(lambda x: filtro_funcao(x, opc), lista))

# por ano
dados_filtrados_por_ano = filtrar_dados(dados, filtro_ano, '2023')
print("Filtrados por ano:", len(dados_filtrados_por_ano))

# por tipo de trabalho
dados_filtrados_por_tipo_trabalho = filtrar_dados(dados, filtro_tipo_trabalho, 'Data Scientist')
print("Filtrados por tipo de trabalho:", dados_filtrados_por_tipo_trabalho)

In [ ]:
# CRIEI APENAS PARA SABERMOS QUAIS PROFISSÕES COM DADOS USAR
cargos_data_science = set()

# Iterar sobre os dicionários e adicionar os valores de 'job_title' contendo "Data Science" ao conjunto
for item in dados:
    if 'job_title' in item and 'Data Science' in item['job_title']:
        cargos_data_science.add(item['job_title'])

print("Cargos contendo 'Data Science':", cargos_data_science)

Cargos contendo 'Data Science': {'Director of Data Science', 'Data Science Consultant', 'Data Science Tech Lead', 'Managing Director Data Science', 'Data Science Lead', 'Data Science Practitioner', 'Data Science Manager', 'Head of Data Science', 'Data Science Director', 'Data Science Engineer'}
